In [118]:
import pandas as pd

In [209]:
df_15076 = pd.read_excel('15076_Agosto22.xlsx', header = 2)
df_18834 = pd.read_excel('18834_Agosto22.xlsx', header = 2)
df_19664 = pd.read_excel('19664_Agosto22.xlsx', header = 2)

In [210]:
columnas = ['RUT-DV', 'NOMBRE', 'UNIDAD', 'CARGO', 'TOTAL HABER']

df_15076 = df_15076[columnas]
df_18834 = df_18834[columnas]
df_19664 = df_19664[columnas]

juntas = pd.concat([df_15076, df_18834, df_19664])
juntas['NOMBRE'] = juntas['NOMBRE'].str.strip()

cambiar_nombres = {'MUÑOZ ALTAMIRANO VIRGINIA DE LAS MERCEDE': 'MUÑOZ ALTAMIRANO VIRGINIA DE LAS MERCEDES',
            'GAETE GAETE JENNIFER PILAR': 'GAETE GAETE JENNIFER DEL PILAR',
            'MARINO LOPEZ MARCO ANTONIO': 'MARINAO LOPEZ MARCO ANTONIO'}

juntas = juntas.replace(cambiar_nombres)

cambiar_cargos = {'10342578-6': 'M422_CARDIOLOGIA',
                '10976333-0': 'M212_MEDICINA INTERNA',
                '12736759-0': 'M200_MEDICO CIRUJANO',
                '13434782-1': 'M200_MEDICO CIRUJANO',
                '14537881-8': 'M200_MEDICO CIRUJANO',
                '15379406-5': 'M200_MEDICO CIRUJANO',
                '16484813-2': 'M200_MEDICO CIRUJANO',
                '19564829-8': '127_TECNICO PARAMEDICO',
                '7877320-0': 'M212_MEDICINA INTERNA',
                '8501857-4': 'M833_JEFE U. DE APOYO',
                '9356279-8': 'M830_JEFE DE SERV. CLINICO'}

juntas = juntas.set_index('RUT-DV')
for rut in cambiar_cargos:
    juntas.loc[rut, 'CARGO'] = cambiar_cargos[rut]

juntas = juntas.reset_index()

In [212]:
suma_haberes_por_unidad = juntas.groupby(by = ['RUT-DV', 'NOMBRE', 'CARGO', 'UNIDAD']).sum()
suma_haberes_por_unidad.to_excel('agrupado_por_cc.xlsx')

suma_haberes_por_persona = juntas.groupby(by = ['RUT-DV', 'NOMBRE', 'CARGO']).sum()
suma_haberes_por_persona.to_excel('agrupado_por_persona.xlsx')

In [213]:
honorarios = pd.read_excel('PERC AGOSTO.xlsx')
columnas = ['RUT-DV', 'NOMBRE', 'UNIDAD O SERVICIO DONDE SE DESEMPEÑA', 'CARGO', 'VALOR TOTAL O BRUTO']
honorarios = honorarios[columnas].rename(columns = {'UNIDAD O SERVICIO DONDE SE DESEMPEÑA': 'UNIDAD', 'VALOR TOTAL O BRUTO': 'TOTAL HABER'})
honorarios = honorarios.set_index('RUT-DV')
honorarios.loc['19296581-0', 'CARGO'] = 'PSICOLOGA 22 HORAS'

juntas['TIPO CONTRATA'] = 1
honorarios['TIPO CONTRATA'] = 2

In [214]:
contrata_y_honorarios = pd.concat([juntas, honorarios])
contrata_y_honorarios['RUT-DV'] = contrata_y_honorarios['RUT-DV'].str.strip().str.upper()

In [217]:
suma_haber_por_persona = contrata_y_honorarios.groupby(by = ['RUT-DV', 'NOMBRE', 'CARGO', 'TIPO CONTRATA']).sum().reset_index()
suma_haber_por_persona.to_excel('suma_haberes_por_persona_con_honorarios.xlsx')

suma_haber_por_persona = contrata_y_honorarios.groupby(by = ['RUT-DV', 'NOMBRE', 'CARGO', 'TIPO CONTRATA', 'UNIDAD']).sum()
suma_haber_por_persona.to_excel('suma_haberes_por_cc_con_honorarios.xlsx')

In [218]:
personas_por_unidad = contrata_y_honorarios.groupby(by = ['RUT-DV', 'NOMBRE', 'UNIDAD']).sum()
mask_personas_que_trabajan_en_mas_de_un_cc = personas_por_unidad.index.get_level_values(0).duplicated(keep = False)
unidades_totales_personas_que_trabajan_en_mas_de_un_cc = personas_por_unidad[mask_personas_que_trabajan_en_mas_de_un_cc].index.get_level_values(2)

In [219]:
unidades_que_quedaran = unidades_totales_personas_que_trabajan_en_mas_de_un_cc[::2]

ruts_a_cambiar_unidad = ['10282072-K', '10976333-0', '12470454-5', '12736759-0', '13507846-8',
       '15188820-8', '15413864-1', '15956991-8', '16093747-5', '16172330-4',
       '16653286-8', '17269121-8', '18018768-5', '18464580-7', '18953398-5',
       '22955669-K', '24591684-1', '6004926-2', '6012866-9', '7259175-5',
       '7877320-0', '8666237-K', '9356279-8', '9381699-4', '9381872-5',
       '9457009-3']

diccionario_cambio_unidades = dict(zip(ruts_a_cambiar_unidad, unidades_que_quedaran))

In [220]:
nueva_contrata = contrata_y_honorarios.copy()
nueva_contrata = nueva_contrata.set_index('RUT-DV')

for rut in diccionario_cambio_unidades:
    nueva_contrata.loc[rut, 'UNIDAD'] = diccionario_cambio_unidades[rut]

nueva_contrata = nueva_contrata.reset_index()

In [221]:
desglose_sin_cc_repetidos = nueva_contrata.groupby(by = ['RUT-DV', 'NOMBRE', 'UNIDAD', 'CARGO']).sum()
desglose_sin_cc_repetidos.to_excel('desglose_sin_cc_repetidos.xlsx')

In [222]:
desglose_sin_cc_repetidos

,,,,TOTAL HABER,TIPO CONTRATA
RUT-DV,NOMBRE,UNIDAD,CARGO,,
10007232-7,CAMPOS PONCE SANDRA EVA,3364_SERVICIO DE MEDICINA BRONCOPULMONAR,127_TECNICO PARAMEDICO,1346536,1
10033369-4,MEDINA SANCHEZ ROXANA SOLEDAD,3530_UNIDAD APOYO ANESTESIA Y PABELLON,127_TECNICO PARAMEDICO,1130457,1
10036715-7,VILLALOBOS RUIZ ARNOLDO ALFONSO,3312_UNIDAD DE MEDICINA INTERMEDIA,130_TECNICO NIVEL SUPERIOR ENFERMERIA,1023328,1
10044862-9,REYES REYES SANDRA BELGICA,3578_ADMISION,10_ENFERMERA,2510150,1
10081465-K,BAEZA MARTINEZ SANDRA MARIANELA,4159_GESTION DE PACIENTES,119_ADMINISTRATIVO,569096,2
...,...,...,...,...,...
9919326-3,VILLEGAS CURIANTE JIMENA C,3578_ADMISION,127_TECNICO PARAMEDICO,929478,1
9920592-K,LAZO MALDONADO CARMEN GLORIA,3312_UNIDAD DE MEDICINA INTERMEDIA,111_AUXILIAR,588598,1
9964171-1,SARMIENTO RIVAS LUIS ALFREDO,1945_DEPARTAMENTO OPERACIONES,111_AUXILIAR,959105,1
